In [1]:
from model import fox_model, losses, dot_prod_attention
from data import data_generation, batch_creator, gp_kernels
from keras.callbacks import ModelCheckpoint
from helpers import helpers, masks
from inference import infer
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow as tf
import numpy as np
import matplotlib 
import time
import keras

Using TensorFlow backend.


In [2]:
save_dir = '/Users/omernivron/Downloads/GPT_fox'

In [3]:
df = np.load('/Users/omernivron/Downloads/fnr.npy')

In [4]:
# 0 = foxes
# 1 = rabbits
# 2 = time
# 3 = foxes_tokens
# 4 = rabbits_tokens (token id 3 means rabbit_ode)

In [5]:
t = df[2::5]
f = df[0::5]; r = df[1::5]
f_token = df[3::5]; r_token = df[4::5] 

In [6]:
pad_pos_tr = t[:80]; f_tr = f[:80]; r_tr = r[:80]; f_token_tr = f_token[:80]; r_token_tr = r_token[:80]
pad_pos_te = t[80:]; f_te = f[80:]; r_te = r[80:]; f_token_te = f_token[80:]; r_token_te = r_token[80:]

In [7]:
pad_pos_tr = np.repeat(pad_pos_tr, 2, axis = 0)
pad_pos_te = np.repeat(pad_pos_te, 2, axis = 0)

In [8]:
tar_tr = np.concatenate((f_tr, r_tr), axis = 0); tar_te = np.concatenate((f_te, r_te), axis = 0)
token_tr = np.concatenate((f_token_tr, r_token_tr), axis = 0); token_te = np.concatenate((f_token_te, r_token_te), axis = 0)


In [9]:
pp = masks.position_mask(pad_pos_tr)
pp_te = masks.position_mask(pad_pos_te)

In [10]:
tok_pos_tr, tim_pos_tr, tar_tr , pos_mask, batch_idx_tr = batch_creator.create_batch_foxes(token_tr, pad_pos_tr, tar_tr, pp, batch_s = 10)

In [20]:
loss_object = tf.keras.losses.MeanSquaredError()
train_loss = tf.keras.metrics.Mean(name='train_loss')
test_loss = tf.keras.metrics.Mean(name='test_loss')
m_tr = tf.keras.metrics.Mean()
m_te = tf.keras.metrics.Mean()

In [21]:
@tf.function
def train_step(token_pos, time_pos, tar, pos_mask):
    '''
    A typical train step function for TF2. Elements which we wish to track their gradient
    has to be inside the GradientTape() clause. see (1) https://www.tensorflow.org/guide/migrate 
    (2) https://www.tensorflow.org/tutorials/quickstart/advanced
    ------------------
    Parameters:
    pos (np array): array of positions (x values) - the 1st/2nd output from data_generator_for_gp_mimick_gpt
    tar (np array): array of targets. Notice that if dealing with sequnces, we typically want to have the targets go from 0 to n-1. The 3rd/4th output from data_generator_for_gp_mimick_gpt  
    pos_mask (np array): see description in position_mask function
    ------------------    
    '''
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]
    combined_mask_tar = masks.create_masks(tar_inp)
    with tf.GradientTape(persistent=True) as tape:
        pred, pred_sig = decoder(token_pos, time_pos, tar_inp, True, pos_mask, combined_mask_tar)
#         print('pred: ')
#         tf.print(pred_sig)

        loss, mse, mask = losses.loss_function(tar_real, pred, pred_sig)


    gradients = tape.gradient(loss, decoder.trainable_variables)
#     tf.print(gradients)
# Ask the optimizer to apply the processed gradients.
    optimizer_c.apply_gradients(zip(gradients, decoder.trainable_variables))
    train_loss(loss)
    m_tr.update_state(mse, mask)
#     b = decoder.trainable_weights[0]
#     tf.print(tf.reduce_mean(b))
    return tar_inp, tar_real, pred, pred_sig, combined_mask_tar

In [22]:
@tf.function
def test_step(token_pos_te, time_pos_te, tar_te, pos_mask_te):
    '''
    
    ---------------
    Parameters:
    pos (np array): array of positions (x values) - the 1st/2nd output from data_generator_for_gp_mimick_gpt
    tar (np array): array of targets. Notice that if dealing with sequnces, we typically want to have the targets go from 0 to n-1. The 3rd/4th output from data_generator_for_gp_mimick_gpt  
    pos_mask_te (np array): see description in position_mask function
    ---------------
    
    '''
    tar_inp_te = tar_te[:, :-1]
    tar_real_te = tar_te[:, 1:]
    combined_mask_tar_te = masks.create_masks(tar_inp_te)
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    pred, pred_sig = decoder(token_pos_te, time_pos_te, tar_inp_te, False, pos_mask_te, combined_mask_tar_te)
#     tf.print(tf.math.reduce_max(pred_sig))
    t_loss, t_mse, t_mask = losses.loss_function(tar_real_te, pred, pred_sig)
    tf.print(t_loss)
    test_loss(t_loss)
    m_te.update_state(t_mse, t_mask)
    return tar_real_te, pred, pred_sig

In [23]:
tf.keras.backend.set_floatx('float64')

In [ ]:
if __name__ == '__main__':
    writer = tf.summary.create_file_writer(save_dir + '/logs/')
    optimizer_c = tf.keras.optimizers.Adam(0.007)
    decoder = fox_model.Decoder(16)
    EPOCHS = 750
    batch_s  = 32
    run = 0; step = 0
    num_batches = int(tar_tr.shape[0] * 4 / batch_s)
#     tf.random.set_seed(1)    
    checkpoint = tf.train.Checkpoint(optimizer = optimizer_c, model = decoder)
    main_folder = "/Users/omernivron/Downloads/GPT_fox/ckpt/check_"
    folder = main_folder + str(run); helpers.mkdir(folder)

    with writer.as_default():
        for epoch in range(EPOCHS):
            start = time.time()

            for batch_n in range(num_batches):
                batch_tok_pos_tr, batch_tim_pos_tr, batch_tar_tr , batch_pos_mask, _ = batch_creator.create_batch_foxes(tok_pos_tr, tim_pos_tr, tar_tr , pos_mask, batch_s= 32)
                # batch_tar_tr shape := 128 X 59 = (batch_size, max_seq_len)
                # batch_pos_tr shape := 128 X 59 = (batch_size, max_seq_len)
                tar_inp, tar_real, pred, pred_sig, combined_mask_tar  = train_step(batch_tok_pos_tr, batch_tim_pos_tr, batch_tar_tr, batch_pos_mask)

                if batch_n % 50 == 0:
#                     batch_tok_pos_te, batch_tim_pos_te, batch_tar_te , batch_pos_mask_te, _ = batch_creator.create_batch_foxes(token_te, pad_pos_te, tar_te, pp_te)
#                     tar_real_te, pred, pred_sig = test_step(batch_tok_pos_te, batch_tim_pos_te, batch_tar_te, batch_pos_mask_te)
                    helpers.print_progress(epoch, batch_n, train_loss.result(), test_loss.result(), m_tr.result())
#                     helpers.tf_summaries(run, step, train_loss.result(), test_loss.result(), m_tr.result(), m_te.result())
#                     checkpoint.save(folder + '/')
                step += 1

            print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

Already exists
Epoch 0 batch 0 train Loss 9778.1563 test Loss 0.0000 with MSE metric 9730.3418
Time taken for 1 epoch: 3.241412878036499 secs

Epoch 1 batch 0 train Loss 7238.7994 test Loss 0.0000 with MSE metric 7495.8036
Time taken for 1 epoch: 2.1934361457824707 secs

Epoch 2 batch 0 train Loss 6139.8840 test Loss 0.0000 with MSE metric 6620.6549
Time taken for 1 epoch: 2.17579984664917 secs

Epoch 3 batch 0 train Loss 5615.7915 test Loss 0.0000 with MSE metric 6848.3023
Time taken for 1 epoch: 2.213737964630127 secs

Epoch 4 batch 0 train Loss 5035.6222 test Loss 0.0000 with MSE metric 7232.3076
Time taken for 1 epoch: 2.173903226852417 secs

Epoch 5 batch 0 train Loss 4400.4060 test Loss 0.0000 with MSE metric 7260.6828
Time taken for 1 epoch: 2.1973352432250977 secs

Epoch 6 batch 0 train Loss 3818.8771 test Loss 0.0000 with MSE metric 7139.3505
Time taken for 1 epoch: 2.173475980758667 secs

Epoch 7 batch 0 train Loss 3377.3323 test Loss 0.0000 with MSE metric 7320.3442
Time tak

Epoch 64 batch 0 train Loss 466.4996 test Loss 0.0000 with MSE metric 4694.6869
Time taken for 1 epoch: 2.1731910705566406 secs

Epoch 65 batch 0 train Loss 459.9390 test Loss 0.0000 with MSE metric 4719.5069
Time taken for 1 epoch: 2.1990132331848145 secs

Epoch 66 batch 0 train Loss 453.5481 test Loss 0.0000 with MSE metric 4721.3711
Time taken for 1 epoch: 2.1838648319244385 secs

Epoch 67 batch 0 train Loss 447.2627 test Loss 0.0000 with MSE metric 4711.5311
Time taken for 1 epoch: 2.1827070713043213 secs

Epoch 68 batch 0 train Loss 441.1226 test Loss 0.0000 with MSE metric 4690.5008
Time taken for 1 epoch: 2.195844888687134 secs

Epoch 69 batch 0 train Loss 435.1559 test Loss 0.0000 with MSE metric 4703.4478
Time taken for 1 epoch: 2.1923351287841797 secs

Epoch 70 batch 0 train Loss 429.3686 test Loss 0.0000 with MSE metric 4700.4374
Time taken for 1 epoch: 2.2219629287719727 secs

Epoch 71 batch 0 train Loss 423.7252 test Loss 0.0000 with MSE metric 4704.2592
Time taken for 1 e

Epoch 128 batch 0 train Loss 241.9888 test Loss 0.0000 with MSE metric 4813.6963
Time taken for 1 epoch: 2.2110350131988525 secs

Epoch 129 batch 0 train Loss 240.2025 test Loss 0.0000 with MSE metric 4812.0421
Time taken for 1 epoch: 2.1772427558898926 secs

Epoch 130 batch 0 train Loss 238.4435 test Loss 0.0000 with MSE metric 4810.4142
Time taken for 1 epoch: 2.187378168106079 secs

Epoch 131 batch 0 train Loss 236.7071 test Loss 0.0000 with MSE metric 4806.8122
Time taken for 1 epoch: 2.1795690059661865 secs

Epoch 132 batch 0 train Loss 234.9964 test Loss 0.0000 with MSE metric 4786.6990
Time taken for 1 epoch: 2.1698381900787354 secs

Epoch 133 batch 0 train Loss 233.3169 test Loss 0.0000 with MSE metric 4798.1238
Time taken for 1 epoch: 2.190953016281128 secs

Epoch 134 batch 0 train Loss 231.6607 test Loss 0.0000 with MSE metric 4815.2159
Time taken for 1 epoch: 2.164705991744995 secs

Epoch 135 batch 0 train Loss 230.0235 test Loss 0.0000 with MSE metric 4800.0179
Time taken f

Epoch 192 batch 0 train Loss 164.7428 test Loss 0.0000 with MSE metric 4742.2826
Time taken for 1 epoch: 2.2510969638824463 secs

Epoch 193 batch 0 train Loss 163.9398 test Loss 0.0000 with MSE metric 4752.6094
Time taken for 1 epoch: 2.34002685546875 secs

Epoch 194 batch 0 train Loss 163.1388 test Loss 0.0000 with MSE metric 4741.7842
Time taken for 1 epoch: 2.359250783920288 secs

Epoch 195 batch 0 train Loss 162.3478 test Loss 0.0000 with MSE metric 4735.2095
Time taken for 1 epoch: 2.54773211479187 secs

Epoch 196 batch 0 train Loss 161.5628 test Loss 0.0000 with MSE metric 4730.7143
Time taken for 1 epoch: 2.5318751335144043 secs

Epoch 197 batch 0 train Loss 160.7925 test Loss 0.0000 with MSE metric 4734.9205
Time taken for 1 epoch: 2.4583370685577393 secs

Epoch 198 batch 0 train Loss 160.0208 test Loss 0.0000 with MSE metric 4722.9138
Time taken for 1 epoch: 2.4237418174743652 secs

Epoch 199 batch 0 train Loss 159.2588 test Loss 0.0000 with MSE metric 4714.9137
Time taken for

In [26]:
tok_pos_tr[6, :]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 2., 2., 2., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [28]:
e = decoder.e1
weights = e.get_weights()[0]

In [31]:
weights[0 ,:]

array([ 0.01198372, -0.022828  ,  0.01366034, -0.04658762,  0.01775511,
       -0.02220659, -0.03331034, -0.02029271])

In [32]:
weights[1 ,:]

array([ 0.01487688, -0.02089084, -0.00441254,  0.04802811, -0.04071961,
        0.04035367, -0.02769675, -0.02957628])

In [33]:
weights[2, :]

array([ 0.00312794, -0.0276023 , -0.05479827, -0.01871443,  0.02734485,
       -0.00082137,  0.00663965,  0.03001904])

In [34]:
weights[3, :]

array([ 0.02947604,  0.00421871,  0.04572925,  0.01271821, -0.01137066,
       -0.04584723, -0.03022694, -0.04152265])

In [35]:
weights[4, :]

array([ 0.0121183 , -0.03513595, -0.01996054,  0.0015047 , -0.05293077,
        0.04234427, -0.02420702,  0.01337569])

In [18]:
pred[3, :]

<tf.Tensor: shape=(148,), dtype=float64, numpy=
array([11.44574197, 11.11333129, 10.78999914, 10.47565778, 10.17020513,
        9.87352594,  9.58549299,  9.30596822,  9.03480393,  8.77184388,
        8.51692443,  8.26987559,  8.03052205,  7.79868417,  7.57417887,
        7.35682054,  7.14642179,  6.94279426,  6.74574929,  6.55509852,
        6.37065451,  6.19223122,  6.01964452,  5.85271255,  5.69125611,
        5.53509901,  5.38406826,  5.23799439,  5.09671158,  4.96005788,
        4.82787529,  4.76290531,  4.69681514,  4.62935489,  4.56219694,
        4.49323823,  4.42996997,  4.36730884,  4.30534293,  4.24414548,
        4.18377688,  4.12428635,  4.06571341,  4.00808917,  3.95143747,
        3.89577578,  3.84111611,  3.78746569,  3.73482768,  3.68320169,
        3.63258426,  3.58296936,  3.53434871,  3.48671211,  3.4400478 ,
        3.39434264,  3.34958238,  3.30575187,  3.26283517,  3.22081579,
        3.17967674,  3.1394007 ,  3.09997013,  3.06136729,  3.0235744 ,
        2.986573

In [88]:
combined_mask_tar[1, :, 44]

<tf.Tensor: shape=(148,), dtype=float32, numpy=
array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>

In [96]:
tar_inp[1, :]

<tf.Tensor: shape=(148,), dtype=float64, numpy=
array([183.        , 170.5377    , 158.71284099, 147.52172089,
       136.95626314, 127.00456081, 117.65142542, 108.87892606,
       100.66690691,  92.99347403,  85.83544469,  79.16875492,
        72.96882289,  67.21086754,  61.87018312,  56.92237156,
        52.34353511,  48.11043257,  44.2006024 ,  40.59245631,
        37.26534692, 183.        ,  23.        ,   5.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
       

In [95]:
tar_real[1, :]

<tf.Tensor: shape=(148,), dtype=float64, numpy=
array([170.5377    , 158.71284099, 147.52172089, 136.95626314,
       127.00456081, 117.65142542, 108.87892606, 100.66690691,
        92.99347403,  85.83544469,  79.16875492,  72.96882289,
        67.21086754,  61.87018312,  56.92237156,  52.34353511,
        48.11043257,  44.2006024 ,  40.59245631,  37.26534692,
       183.        ,  23.        ,   5.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
       

In [90]:
pred[8, :]

<tf.Tensor: shape=(148,), dtype=float64, numpy=
array([-0.03583945, -0.03630679, -0.03658146, -0.03666347, -0.03655284,
       -0.0362496 , -0.03575379, -0.03506542, -0.03418455, -0.0331112 ,
       -0.03184543, -0.03038729, -0.02873682, -0.02689408, -0.02485913,
       -0.02263204, -0.02021287, -0.01760169, -0.01479858, -0.01180361,
       -0.01453558, -0.01682202, -0.01887398, -0.02066269, -0.02222955,
       -0.02360825, -0.02482641, -0.02590686, -0.02686857, -0.02772741,
       -0.02849675, -0.02918788, -0.02981041, -0.03037256, -0.03088137,
       -0.03134291, -0.03176244, -0.03214451, -0.0324931 , -0.0328117 ,
       -0.03310334, -0.03337072, -0.0336162 , -0.03384188, -0.03404962,
       -0.03424107, -0.0344177 , -0.03458083, -0.03473165, -0.03487121,
       -0.03500047, -0.03512027, -0.0352314 , -0.03533456, -0.03543038,
       -0.03551943, -0.03560224, -0.03567929, -0.03575101, -0.03581778,
       -0.03587999, -0.03593796, -0.03589755, -0.03584261, -0.03577452,
       -0.035694

In [75]:
batch_pos_mask.shape

(32, 148, 149, 149)

In [41]:
tar_real_te[30, :]

<tf.Tensor: shape=(189,), dtype=float64, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])>

In [43]:
pred[1, :]

<tf.Tensor: shape=(189,), dtype=float64, numpy=
array([-0.04683568, -0.04683568, -0.04683568, -0.04683568, -0.04683568,
       -0.04683568, -0.04683568, -0.04683568, -0.04683568, -0.04683568,
       -0.04683568, -0.04683568, -0.04683568, -0.04683568, -0.04683568,
       -0.04683568, -0.04683568, -0.04683568, -0.04683568, -0.04683568,
       -0.04683568, -0.04683568, -0.04683568, -0.04683568, -0.04683568,
       -0.04683568, -0.04683568, -0.04683568, -0.04683568, -0.04683568,
       -0.04683568, -0.04683568, -0.04683568, -0.04683568, -0.04683568,
       -0.04683568, -0.04683568, -0.04683568, -0.04683568, -0.04683568,
       -0.04683568, -0.04683568, -0.04683568, -0.04683568, -0.04683568,
       -0.04683568, -0.04683568, -0.04683568, -0.04683568, -0.04683568,
       -0.04683568, -0.04683568, -0.04683568, -0.04683568, -0.04683568,
       -0.04683568, -0.04683568, -0.04683568, -0.04683568, -0.04683568,
       -0.04683568, -0.04683568, -0.04683568, -0.04683568, -0.04683568,
       -0.046835

In [46]:
pred_sig[2, :]

<tf.Tensor: shape=(189,), dtype=float64, numpy=
array([2.79753801, 2.79753801, 2.79753801, 2.79753801, 2.79753801,
       2.79753801, 2.79753801, 2.79753801, 2.79753801, 2.79753801,
       2.79753801, 2.79753801, 2.79753801, 2.79753801, 2.79753801,
       2.79753801, 2.79753801, 2.79753801, 2.79753801, 2.79753801,
       2.79753801, 2.79753801, 2.79753801, 2.79753801, 2.79753801,
       2.79753801, 2.79753801, 2.79753801, 2.79753801, 2.79753801,
       2.79753801, 2.79753801, 2.79753801, 2.79753801, 2.79753801,
       2.79753801, 2.79753801, 2.79753801, 2.79753801, 2.79753801,
       2.79753801, 2.79753801, 2.79753801, 2.79753801, 2.79753801,
       2.79753801, 2.79753801, 2.79753801, 2.79753801, 2.79753801,
       2.79753801, 2.79753801, 2.79753801, 2.79753801, 2.79753801,
       2.79753801, 2.79753801, 2.79753801, 2.79753801, 2.79753801,
       2.79753801, 2.79753801, 2.79753801, 2.79753801, 2.79753801,
       2.79753801, 2.79753801, 2.79753801, 2.79753801, 2.79753801,
       2.79753